#  GeoLocation Map - Gamma EBR ratings + Flodability Scores + LoanBook

In [1]:
import pandas
print(pandas.__version__)

0.24.2


In [2]:
import numpy as np 
import pandas as pd 
import folium
from folium.plugins import Search
from functools import reduce
import math

df_gamma_loan = pd.read_csv('data/input/gamma_loanbook_published/part-00000-9c4e7859-a054-4410-853e-3bd2b31b23c1-c000-attempt_20230207145556716774276274426454_0008_m_000000_8.csv')

In [3]:
df_gamma_loan.describe()

,ecad_id,building_id,address_line_7,address_line_8,address_line_9,etrs89_lat,etrs89_long,ber_rating_kwh,co2_emission,riverrp,...,Rate/Margin,Initial Borrow capital,Loan to Value (LTV),Monthly Repayment,Monthly_Capital_Repayment,Months_Paid,Months_Due,Property_Valuation,Initial_Deposit,Mortgage Outstanding Balance
count,2.553800e+04,2.553800e+04,0.0,0.0,0.0,25538.000000,25538.000000,25538.000000,25538.000000,25538.000000,...,25538.000000,25538.000000,25538.000000,25538.000000,25538.000000,25538.000000,25538.000000,2.553800e+04,25538.000000,25538.000000
mean,1.701600e+09,1.400867e+09,NaN,NaN,NaN,54.216207,-8.567754,220.685802,53.987362,7.467499,...,0.073073,251124.799906,0.750337,1732.093556,697.574722,310.970789,49.029211,3.346517e+05,83526.872332,39420.697587
std,2.549289e+05,6.863783e+05,NaN,NaN,NaN,0.106643,0.182830,67.592197,19.525710,78.506486,...,0.001618,110467.458330,0.006264,778.890689,306.854054,22.641157,22.641157,1.470783e+05,36742.700255,31319.245860
min,1.700015e+09,1.400055e+09,NaN,NaN,NaN,53.927076,-9.131647,-16.954646,-5.091399,0.000000,...,0.053000,60000.000000,0.750000,403.220000,166.670000,84.000000,11.000000,8.000000e+04,20000.000000,1830.760000
25%,1.701585e+09,1.400226e+09,NaN,NaN,NaN,54.155385,-8.598852,176.000000,41.000000,0.000000,...,0.072000,155763.750000,0.750000,1057.315000,432.680000,301.000000,35.000000,2.076850e+05,51731.250000,15142.437500
50%,1.701595e+09,1.400666e+09,NaN,NaN,NaN,54.254852,-8.501129,214.000000,52.000000,0.000000,...,0.073000,250732.500000,0.750000,1718.955000,696.485000,313.000000,47.000000,3.342700e+05,83437.500000,32837.205000
75%,1.701602e+09,1.401715e+09,NaN,NaN,NaN,54.273464,-8.467546,261.000000,66.000000,0.000000,...,0.074000,346166.250000,0.750000,2395.125000,961.580000,325.000000,59.000000,4.613950e+05,115268.750000,56916.512500
max,1.702376e+09,1.402038e+09,NaN,NaN,NaN,54.470316,-8.175443,840.000000,238.000000,1000.000000,...,0.075000,760000.000000,0.900000,4654.770000,2111.120000,349.000000,276.000000,1.013333e+06,253333.330000,582665.920000


In [4]:
df_gamma_loan.head(10)

,ecad_id,building_id,address_line_1,address_line_2,address_line_3,address_line_4,address_line_5,address_line_6,address_line_7,address_line_8,...,Initial Borrow capital,Loan to Value (LTV),Monthly Repayment,Next_Payment_Due,Monthly_Capital_Repayment,Months_Paid,Months_Due,Property_Valuation,Initial_Deposit,Mortgage Outstanding Balance
0,1700015484,1400072397,31 CHAPEL STREET,SLIGO,F91 K402,NaN,NaN,NaN,NaN,NaN,...,250000,0.80,1388.27,01/02/2023,694.45,84,276,312500.0000,62500.00,191666.20
1,1700015488,1400072894,22 MAIL COACH ROAD,SLIGO,F91 YPY7,NaN,NaN,NaN,NaN,NaN,...,300000,0.90,1837.41,01/02/2023,833.34,84,276,333333.3333,33333.33,229999.44
2,1700019562,1401374452,CHAPLAIN'S RESIDENCE,ATLANTIC TECHNOLOGICAL UNIVERSITY,CLOGHEREVAGH,CO. SLIGO,F91 XW98,NaN,NaN,NaN,...,365000,0.85,2271.17,01/02/2023,1013.89,84,276,429411.7647,64411.76,279833.24
3,1700030305,1401795485,BARRACK ROAD,COLLOONEY,CO. SLIGO,F91 D780,NaN,NaN,NaN,NaN,...,400000,0.75,2221.22,01/02/2023,1111.12,84,276,533333.3333,133333.33,306665.92
4,1700031632,1401731665,MOLLAGH,CLOONACOOL,CO. SLIGO,F91 WD68,NaN,NaN,NaN,NaN,...,250000,0.80,1531.18,01/02/2023,694.45,84,276,312500.0000,62500.00,191666.20
5,1700040544,1401768705,RATHLEE,EASKEY,CO. SLIGO,F26 NP74,NaN,NaN,NaN,NaN,...,450000,0.90,2800.07,01/02/2023,1250.01,84,276,500000.0000,50000.00,344999.16
6,1700041125,1400071618,2 PATTON HOUSE,OAKFIELD ROAD,BALLYDOOGAN,SLIGO,F91 AE84,NaN,NaN,NaN,...,250000,0.85,1388.27,01/02/2023,694.45,84,276,294117.6471,44117.65,191666.20
7,1700042473,1400655947,CLARAGH IRISH,LAVAGH,BALLYMOTE,CO. SLIGO,F56 YY89,NaN,NaN,NaN,...,250000,0.75,1531.18,01/02/2023,694.45,84,276,333333.3333,83333.33,191666.20
8,1700049294,1400075563,23 OLD FORT ROAD,KEVINSFORT,SLIGO,F91 XD80,NaN,NaN,NaN,NaN,...,600000,0.80,3733.42,01/02/2023,1666.67,84,276,750000.0000,150000.00,459999.72
9,1700049295,1400264459,4 MULBERRY PARK,BALLINODE,SLIGO,F91 RW01,NaN,NaN,NaN,NaN,...,250000,0.90,1388.27,01/02/2023,694.45,84,276,277777.7778,27777.78,191666.20


In [21]:
df_gamma_loan.dtypes

ecad_id                                  int64
building_id                              int64
address_line_1                          object
address_line_2                          object
address_line_3                          object
address_line_4                          object
address_line_5                          object
address_line_6                          object
address_line_7                         float64
address_line_8                         float64
address_line_9                         float64
etrs89_lat                             float64
etrs89_long                            float64
ber_rating                              object
ber_rating_kwh                         float64
co2_emission                           float64
riverrp                                  int64
rmax20                                 float64
rmax75                                 float64
rmax100                                float64
rmax200                                float64
rmax1000     

In [22]:
list_of_variables = ['ecad_id','ber_rating_kwh','co2_emission','Asset_id','Property_Valuation','Initial_Deposit','Mortgage Outstanding Balance']

df_loan_id_ecad_id_mtg = df_gamma_loan[list_of_variables].copy()

In [23]:
df_loan_id_ecad_id_mtg.dtypes

ecad_id                           int64
ber_rating_kwh                  float64
co2_emission                    float64
Asset_id                          int64
Property_Valuation              float64
Initial_Deposit                 float64
Mortgage Outstanding Balance    float64
dtype: object

In [24]:
df_loan_id_ecad_id_mtg.head(2)

,ecad_id,ber_rating_kwh,co2_emission,Asset_id,Property_Valuation,Initial_Deposit,Mortgage Outstanding Balance
0,1700015484,254.0,59.0,10000001,312500.0000,62500.00,191666.20
1,1700015488,311.0,77.0,10000002,333333.3333,33333.33,229999.44


In [25]:
df_loan_id_ecad_id_mtg.describe()

,ecad_id,ber_rating_kwh,co2_emission,Asset_id,Property_Valuation,Initial_Deposit,Mortgage Outstanding Balance
count,2.553800e+04,25538.000000,25538.000000,2.553800e+04,2.553800e+04,25538.000000,25538.000000
mean,1.701600e+09,220.685802,53.987362,1.001277e+07,3.346517e+05,83526.872332,39420.697587
std,2.549289e+05,67.592197,19.525710,7.372330e+03,1.470783e+05,36742.700255,31319.245860
min,1.700015e+09,-16.954646,-5.091399,1.000000e+07,8.000000e+04,20000.000000,1830.760000
25%,1.701585e+09,176.000000,41.000000,1.000639e+07,2.076850e+05,51731.250000,15142.437500
50%,1.701595e+09,214.000000,52.000000,1.001277e+07,3.342700e+05,83437.500000,32837.205000
75%,1.701602e+09,261.000000,66.000000,1.001915e+07,4.613950e+05,115268.750000,56916.512500
max,1.702376e+09,840.000000,238.000000,1.002554e+07,1.013333e+06,253333.330000,582665.920000


In [26]:
df_gamma_loan.tail(1)

,ecad_id,building_id,address_line_1,address_line_2,address_line_3,address_line_4,address_line_5,address_line_6,address_line_7,address_line_8,...,Initial Borrow capital,Loan to Value (LTV),Monthly Repayment,Next_Payment_Due,Monthly_Capital_Repayment,Months_Paid,Months_Due,Property_Valuation,Initial_Deposit,Mortgage Outstanding Balance
25537,1702375679,1400072628,6A CONNOLLY STREET,SLIGO,F91 WNV0,NaN,NaN,NaN,NaN,NaN,...,441345,0.75,3085.95,01/02/2023,1225.96,289,71,588460.0,147115.0,87042.56


# Map Vizualisation creation from Pandas DataFrame using Leaflet and Folium library

In [10]:
# Increase display in Jupyter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [11]:
df_gamma_loan.dtypes

ecad_id                                  int64
building_id                              int64
address_line_1                          object
address_line_2                          object
address_line_3                          object
address_line_4                          object
address_line_5                          object
address_line_6                          object
address_line_7                         float64
address_line_8                         float64
address_line_9                         float64
etrs89_lat                             float64
etrs89_long                            float64
ber_rating                              object
ber_rating_kwh                         float64
co2_emission                           float64
riverrp                                  int64
rmax20                                 float64
rmax75                                 float64
rmax100                                float64
rmax200                                float64
rmax1000     

In [12]:
df_gamma_loan.shape

(25538, 73)

In [13]:
lat= 54.26969
lang= -8.46943

In [14]:
#Color codes for BER rating scale
colors_ber={
    'A1':'#00a54f',
    'A2':'#00a54f',
    'A3':'#00a54f',
    'B1':'#4cb848',
    'B2':'#4cb848',
    'B3':'#4cb848',
    'C1':'#bed630',
    'C2':'#bed630',
    'C3':'#bed630',
    'D1':'#fff101',
    'D2':'#fff101',
    'E1':'#fcb814',
    'E2':'#fcb814',
    'F':'#f36e21',
    'G':'#ee1d23',
}

In [15]:
#Color codes for flood rating scale
colors_flood={
    'No Colour':'#deebf6',
    'green':'#a5d45d',
    'amber':'#ecc63a',
    'red':'#d92424',
    'black 1':'#919191',
    'black 2':'#242424',
}

In [16]:
# Creating the popup labels for energy rating data

def popup_html_ber(row):
    i = row
    ecad_id=df_gamma_loan['ecad_id'].iloc[i] 
    Address= df_gamma_loan['address_line_1'].iloc[i] +' '+ df_gamma_loan['address_line_2'].iloc[i] +' '+ df_gamma_loan['address_line_3'].iloc[i]
    Ber_Rating = df_gamma_loan['ber_rating'].iloc[i]
    Ber_Rating_Kwh = df_gamma_loan['ber_rating_kwh'].iloc[i]
    Co2_Emission = df_gamma_loan['co2_emission'].iloc[i]

    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px">ECAD ID:{}</h4>""".format(ecad_id) + """</center>
<center> <table style="height: 126px; width: 305px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Address </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">"""+ Address + """</td>
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Ber Rating </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Ber_Rating) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Ber Rating KWh </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{} kWh/m\u00b2/year</td>""".format(Ber_Rating_Kwh) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">CO\u2082 Emission </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Co2_Emission) + """
</tr>
</tbody>
</table></center>
</html>
"""
    return html

In [17]:
# Creating the popup labels for flood rating data

def popup_html_flood(row):
    i = row
    ecad_id=df_gamma_loan['ecad_id'].iloc[i] 
    address= df_gamma_loan['address_line_1'].iloc[i] +' '+ df_gamma_loan['address_line_2'].iloc[i] +' '+ df_gamma_loan['address_line_3'].iloc[i]
    river_flooding_first = df_gamma_loan['riverrp'].iloc[i]
    coastal_flooding_first = df_gamma_loan['coastaludrp'].iloc[i]
    surf_water_flooding_first = df_gamma_loan['swaterrp'].iloc[i]
    
    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px">ECAD ID:{}</h4>""".format(ecad_id) + """</center>
<center> <table style="height: 130px; width: 500px;">
<tbody>
<tr>
<td style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Address </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">"""+ address + """</td>
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Return Period at which River Flooding First Occurred </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(river_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Return Period at which Coastal Flooding First Occurred  </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(coastal_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Return Period at which River Surface Water Flooding First Occurred  </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(surf_water_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Combined Score - Undefended River, Coastal and Surface Water</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['floodscore_ud'].iloc[i]) + """
</tr>
</tbody>
</table>
</center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">River</h5>
<center> <table style="height: 130px; width: 500px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">20 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">75 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">100 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">200 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">1000 years </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Max. Depth of Flooding in the River Undefended in Meters </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['rmax20'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['rmax75'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['rmax100'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['rmax200'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['rmax1000'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">River Undefended Flood Score </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['r20matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['r75matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['r100matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['r200matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['r1000matrix'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property Modelled for Undefended River Flood </span></td>
<td colspan="5" style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['model_river'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Coastal</h5>
<center> <table style="height: 130px; width: 500px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">75 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">100 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">200 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">1000 years </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Max. Depth of Flooding in the Coastal Undefended in Meters </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cudmax75'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cudmax100'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cudmax200'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cudmax1000'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Coastal Undefended Flood Score </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cud75matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cud100matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cud200matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cud1000matrix'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property Modelled for Coastal Flood </span></td>
<td colspan="4" style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['model_coastal'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Surface Water</h5>
<center> <table style="height: 130px; width: 500px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">75 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">200 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">1000 years </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Max. Depth of Flooding in the Surface Water Undefended in Meters </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['swmax75'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['swmax200'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['swmax1000'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Surface Water Undefended Flood Score </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['sw75matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['sw200matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['sw1000matrix'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property Modelled for Surface Water Flood </span></td>
<td colspan="4" style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['model_sw'].iloc[i]) + """
</tr>
</tbody>
</table> </center>
</html>
""" 
    return html

In [18]:
# Creating the popup labels for gamma and loan data

def popup_html_index(row):
    i = row
    ecad_id=df_gamma_loan['ecad_id'].iloc[i] 
    address= df_gamma_loan['address_line_1'].iloc[i] +' '+ df_gamma_loan['address_line_2'].iloc[i] +' '+ df_gamma_loan['address_line_3'].iloc[i]
    river_flooding_first = df_gamma_loan['riverrp'].iloc[i]
    coastal_flooding_first = df_gamma_loan['coastaludrp'].iloc[i]
    surf_water_flooding_first = df_gamma_loan['swaterrp'].iloc[i]
    
    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px">ECAD ID:{}</h4>""".format(ecad_id) + """</center>
<center> <table style="height: 130px; width: 600px;">
<tbody>
<tr>
<td style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Address </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">"""+ address + """</td>
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">BER Rating </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} ({} kWh/m\u00b2/year)</td>""".format(df_gamma_loan['ber_rating'].iloc[i],df_gamma_loan['ber_rating_kwh'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">CO\u2082 Emission</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['co2_emission'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Return Period at which River Flooding First Occurred </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(river_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Return Period at which Coastal Flooding First Occurred  </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(coastal_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Return Period at which River Surface Water Flooding First Occurred  </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(surf_water_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Combined Score - Undefended River, Coastal and Surface Water</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['floodscore_ud'].iloc[i]) + """
</tr>
</tbody>
</table>
</center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">River</h5>
<center> <table style="height: 130px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">20 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">75 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">100 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">200 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">1000 years </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Max. Depth of Flooding in the River Undefended in Meters </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['rmax20'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['rmax75'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['rmax100'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['rmax200'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['rmax1000'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">River Undefended Flood Score </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['r20matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['r75matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['r100matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['r200matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['r1000matrix'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Coastal</h5>
<center> <table style="height: 130px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">75 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">100 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">200 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">1000 years </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Max. Depth of Flooding in the Coastal Undefended in Meters </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cudmax75'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cudmax100'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cudmax200'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cudmax1000'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Coastal Undefended Flood Score </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cud75matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cud100matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cud200matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['cud1000matrix'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Surface Water</h5>
<center> <table style="height: 130px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">75 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">200 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">1000 years </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Max. Depth of Flooding in the Surface Water Undefended in Meters </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['swmax75'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['swmax200'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['swmax1000'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Surface Water Undefended Flood Score </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['sw75matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['sw200matrix'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['sw1000matrix'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Loan Details</h5>
<center> <table style="height: 130px; width:600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Asset_id</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Program/product</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Term</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Rate/margin</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Disbursment Date</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Completion Date</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Initial Borrow capital</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Loan to Value</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Monthly repayment</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Mortgage Outstanding Balance</span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['Asset_id'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['Program/product'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['MTG Term'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}%</td>""".format(round(df_gamma_loan['Rate/Margin'].iloc[i]*100,2)) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['Disbursment Date ( start MTG Date)'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_gamma_loan['Completion Date ( Closing MTG Date)'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">€{}</td>""".format(df_gamma_loan['Initial Borrow capital'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}%</td>""".format(math.floor(df_gamma_loan['Loan to Value (LTV)'].iloc[i]*100)) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">€{}</td>""".format(df_gamma_loan['Monthly Repayment'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">€{}</td>""".format(df_gamma_loan['Mortgage Outstanding Balance'].iloc[i]) + """
</tr>
</tbody>
</table> </center>
</html>
""" 
    return html

In [19]:
#Creating lagends

def add_legend(maps, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_label = dict(zip(labels, colors))
    
    legends = ""     
    for label, color in color_label.items():
        legends += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='legend' class='legend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legends}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var Layout = (function() {{
                    var flag = false;
                    return function() {{
                        if (!flag) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!flag)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          flag = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        Layout()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .legend {
        z-index:9999;
        float:right;
        background-color: rgb(255, 255, 255);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .legend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .legend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .legend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .legend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .legend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .legend a {
        color: #777;
        }
    </style>
    """

    maps.get_root().header.add_child(folium.Element(script + css))

    return maps

In [20]:
#Plotting BER_rating data

sligo_ber=folium.Map(location=[lat,lang],zoom_start=10)

for d in df_gamma_loan.iterrows():
        html = popup_html_ber(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        folium.CircleMarker(
                    [d[1]["etrs89_lat"], d[1]["etrs89_long"]],
                    radius=6,
                    color=colors_ber[d[1]["ber_rating"]],
                    fill=True,
                    fill_color=colors_ber[d[1]["ber_rating"]],
                    fill_opacity=0.7,
                    popup=popup
            ).add_to(sligo_ber)

sligo_ber = add_legend(sligo_ber, 'Building Energy Rating', colors = list(colors_ber.values()), labels = list(colors_ber.keys()))
sligo_ber.save('data/output/visualization_gamma_loanbook/Sligo_Energy_Rating.html')

# Display Optional
# sligo_ber


In [21]:
#Plotting flood_rating data

sligo_flood=folium.Map(location=[lat,lang],zoom_start=10)

for d in df_gamma_loan.iterrows():
        html = popup_html_flood(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        folium.CircleMarker(
                    [d[1]["etrs89_lat"], d[1]["etrs89_long"]],
                    radius=6,
                    color=colors_flood[d[1]["floodability_index_ud"]],
                    fill=True,
                    fill_color=colors_flood[d[1]["floodability_index_ud"]],
                    fill_opacity=0.7,
                    popup=popup
            ).add_to(sligo_flood)

sligo_flood = add_legend(sligo_flood, 'Likelihood of flooding', colors = list(colors_flood.values()), labels = ('Very Low', 'Low', 'Moderate', 'Moderate to High', 'High', 'Very High'))
sligo_flood.save('data/output/visualization_gamma_loanbook/Sligo_Flood_Rating.html')

# Display Optional 
#sligo_flood


# Display - Gamma & Loanbook  search by index ecad_id - Ireland Sligo County

In [ ]:
#Plotting gamma_loan data - index search

Sligo_gamma_loan=folium.Map(location=[lat,lang],zoom_start=10)
from folium import plugins
cluster=plugins.MarkerCluster().add_to(Sligo_gamma_loan)

for d in df_gamma_loan.iterrows(): 
        html = popup_html_index(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=600)
        folium.Marker(location=[d[1]["etrs89_lat"], d[1]["etrs89_long"]], popup=popup,name=d[1]["ecad_id"]).add_to(cluster)
Search(cluster,search_label='name',placeholder='Search for ECAD ID').add_to(Sligo_gamma_loan)            
Sligo_gamma_loan.save('data/output/visualization_gamma_loanbook/Sligo_Gamma_Loan_IndexSearch.html')

# Display Optional
# Sligo_gamma_loan

# Display - Gamma & Loanbook  search by index ecad_id - Ireland Sligo County

In [ ]:
exit()